# Shallow Neural Network Implementation

## Importing Required Libraries

First, import the necessary libraries. Note that in this assignment, you are only allowed to use the libraries provided in the notebook.

In [2]:
import numpy as np
import pandas as pd

## Dataset

In this exercise, we will use the simple yet famous **Pima Indians Diabetes** dataset. This dataset includes information from **768 Native American women** from the Pima tribe, collected to examine the risk factors for developing type 2 diabetes. The data includes age, weight, height, family history of diabetes, blood pressure, blood glucose levels, and other factors.

<center>
<div style="line-height:200%; font-size:medium">
    
| Column | Description |
|:------:|:-----------:|
|Pregnancies|Number of pregnancies|
|Glucose|Blood glucose level (mg/dL)|
|BloodPressure|Systolic blood pressure (mmHg)|
|SkinThickness|Skin thickness (mm)|
|Insulin|Blood insulin level (μU/mL)|
|BMI|Body mass index (kg/m²)|
|DiabetesPedigreeFunction|Function representing family history of diabetes|
|Age|Age of the woman (years)|
|Outcome|Non-diabetic (0) or diabetic (1)|

</div>
</center>

### Reading the Dataset

First, you need to read the dataset file. You can read the training data from the file `diabetes_train.csv` located in the `data` folder and use the samples in it to train the model. The model's performance will be evaluated on `diabetes_test.csv`, which has the same structure as the training data except that the `Outcome` column is removed.

In [15]:
train_data = pd.read_csv("diabetes_train(1).csv")
train_data.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [16]:
test_data = pd.read_csv("diabetes_test(1).csv")
test_data.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age
0,6,98,58,33,190,34.0,0.430,43
1,9,154,78,30,100,30.9,0.164,45
2,6,165,68,26,168,33.6,0.631,49
3,1,99,58,10,0,25.4,0.551,21
4,10,68,106,23,49,35.5,0.285,47


## Preprocessing and Feature Engineering

First, store the target variable column (`Outcome`) in a separate DataFrame and then remove this column from the `train_data` DataFrame to create the equivalent matrices $X$ and $y$.

In [17]:
train_data_outcome = train_data["Outcome"]
train_data = train_data.drop(["Outcome"], axis=1)

train_data.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age
0,6,148,72,35,0,33.6,0.627,50
1,1,85,66,29,0,26.6,0.351,31
2,8,183,64,0,0,23.3,0.672,32
3,1,89,66,23,94,28.1,0.167,21
4,0,137,40,35,168,43.1,2.288,33


One of the crucial preprocessing steps is feature scaling to a normal distribution, commonly referred to as normalization. Normalization helps reduce significant weight fluctuations and accelerates model convergence. In this assignment, you should normalize each feature so that their mean is `0` and their variance is `1`. This can be done using the following formula:

For a data series `X = [x_1, x_2, ..., x_n]`, subtract the mean from each data sample (`x_i`) and divide by the standard deviation (sigma) to obtain the normalized data series.

$$ Z = \frac{x_i - \bar{x}}{\sigma} $$

**Note:** Since we only have access to the training data when building the model, use the mean and standard deviation from the training samples to normalize the test samples as well.

In [8]:
for column in train_data.columns:
    mean = train_data[column].mean()
    std = train_data[column].std()
    train_data[column] = (train_data[column] - mean) / std
    test_data[column] = (test_data[column] - mean) / std

train_data.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age
0,0.649833,0.854539,0.166518,0.900880,-0.687695,0.222281,0.438405,1.443781
1,-0.835754,-1.096441,-0.140758,0.526362,-0.687695,-0.672046,-0.370035,-0.178571
2,1.244068,1.938416,-0.243184,-1.283807,-0.687695,-1.093658,0.570216,-0.093184
3,-0.835754,-0.972569,-0.140758,0.151844,0.123855,-0.480405,-0.908995,-1.032441
4,-1.132872,0.513891,-1.472290,0.900880,0.762734,1.436011,5.303692,-0.007797


Next, add a bias term to the DataFrame. To do this, add a column with a value of `1` at the beginning of the dataset.

In [10]:
train_bias = np.array(1)
train_data.insert(loc=0, column="train_bias", value=train_bias)

test_bias = np.array(1)
test_data.insert(loc=0, column="test_bias", value=test_bias)

train_data.head()

,train_bias,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age
0,1,0.649833,0.854539,0.166518,0.900880,-0.687695,0.222281,0.438405,1.443781
1,1,-0.835754,-1.096441,-0.140758,0.526362,-0.687695,-0.672046,-0.370035,-0.178571
2,1,1.244068,1.938416,-0.243184,-1.283807,-0.687695,-1.093658,0.570216,-0.093184
3,1,-0.835754,-0.972569,-0.140758,0.151844,0.123855,-0.480405,-0.908995,-1.032441
4,1,-1.132872,0.513891,-1.472290,0.900880,0.762734,1.436011,5.303692,-0.007797


Before designing and training the model, convert the datasets from DataFrames to NumPy arrays. Therefore, in this step, convert the DataFrames `train_data` and `train_data_outcome` to NumPy arrays. Additionally, use the `train_test_split` function to split this dataset into training and validation sets with a ratio of `0.2`.

**Note:** According to previous lectures, each **row** of the input matrix represents a **feature**, and each **column** represents a **sample**. Therefore, you need to transpose the feature matrix. This step should also be applied to the target variable (`train_data_outcome`).

In [13]:
from sklearn.model_selection import train_test_split

X_train, X_validation, y_train, y_validation = train_test_split(train_data, train_data_outcome, test_size=0.2)

X_train = X_train.T
X_validation = X_validation.T
y_train = y_train.T
y_validation = y_validation.T
test_data_numpy = test_data.T

To ensure the correctness of input and output settings, running the next cell should produce the following output:

```
X_train.shape:(9, 534), y_train.shape:(534,)
X_validation.shape:(9, 134), y_validation.shape:(134,)
test_data_numpy.shape:(9, 100)
```

In [14]:
print(f'X_train.shape:{X_train.shape}, y_train.shape:{y_train.shape}')
print(f'X_validation.shape:{X_validation.shape}, y_validation.shape:{y_validation.shape}')
print(f'test_data_numpy.shape:{test_data_numpy.shape}')

X_train.shape:(9, 534), y_train.shape:(534,)
X_validation.shape:(9, 134), y_validation.shape:(134,)
test_data_numpy.shape:(9, 100)


## Modeling

Now that the data is processed and ready, it's time for the main part—building the model. You are required to implement a simple shallow neural network using gradient descent from scratch. We will explain each component of this model step by step to guide you through its implementation.

This model is a shallow neural network with one hidden layer containing `1000` neurons. The activation function for this layer is the Rectified Linear Unit (ReLU), which you are familiar with from the activation function lectures. The activation function for the output layer is the sigmoid function. Note that the required formulas for each part are provided below. We have also implemented the two activation functions for you.

```python
sigmoid_Z = 1 / (1 + np.exp(-Z))
```

```python
ReLU_Z = np.maximum(0, Z)
```

**Note:** Only use the NumPy library for mathematical operations and computations, and define your lists as NumPy arrays.

### Reminder: Sigmoid Function

| Sigmoid Function | Derivative of Sigmoid Function |
| :---: | :--: |
| $f(z) = \frac{1}{1 + e^{-z}}$ | $f'(z) = f(z)(1-f(z))$ |

### Reminder: ReLU Activation Function

| ReLU Function  | Derivative of ReLU Function  |
| :---: | :--: |
|$$f(z) = \begin{cases} 0 & \text{if } z < 0 \\ z & \text{if } z \geq 0\end{cases}$$|$$f'(z) = \begin{cases} 0 & \text{if } z < 0 \\ 1 & \text{if } z \geq 0\end{cases}$$|

### `Model` Class Construction

Create a class named `Model` that contains the following three methods. We will explain each method in detail below.

In [ ]:
def __init__(self)
def predict(self, inputs)
def update_weights_for_one_epoch(self, inputs, outputs, learning_rate)
def fit(self, inputs, outputs, learning_rate, epochs=64)

#### `__init__` Method

In the `__init__(self)` method, initialize the weights of the hidden and output layers (`w1` and `w2`) randomly with a mean of `0` and a standard deviation of `0.01`. You can use the `np.random.randn` function for this purpose. Note that `np.random.randn` generates random numbers with a mean of `0` and a standard deviation of `1`, so you need to adjust these values accordingly to meet the problem requirements.

#### `predict` Method

The `predict(self, inputs)` method takes the inputs and sequentially returns the outputs of both layers (`A_1` and `A_2`). Implement this process according to the following formulas:

$$Z^{[1]}=W^{[1]}.X$$
$$A^{[1]}=ReLU(Z^{[1]})$$
$$Z^{[2]}=W^{[2]}A^{[1]}$$
$$A^{[2]}=\sigma(Z^{[2]})=\frac{1}{1+e^{-Z^{[2]}}}=Y_{pred}$$

**Hint:** To perform matrix multiplication between two matrices, use the `arr1.dot(arr2)` function. For example, the formula $Z^{[1]}=W^{[1]}X$ corresponds to `W_1.dot(X)` in Python. Alternatively, you can use the `@` operator as `W_1 @ X`.

#### `update_weights_for_one_epoch` Method

In the `update_weights_for_one_epoch(self, inputs, outputs, learning_rate)` method, update the network's weights for one epoch. Note that `learning_rate` is the learning rate or alpha. The required formulas for this section are provided below. In the next chapter, we will explain in detail how to compute them.

**Weight Update for `w2`:**

$$W^{[2]} = W^{[2]} + \Delta W^{[2]}$$
$$\Delta W^{[2]} = - \alpha \frac{\partial cost}{\partial W^{[2]}}$$
$$\frac{\partial cost}{\partial W^{[2]}} = \left(\frac{-2}{n}(Y_{true}-A^{[2]}) \odot A^{[2]} \odot (1-A^{[2]})\right) \bullet A^{[1]T}$$
$$W^{[2]} = W^{[2]} + \left(\frac{2 \alpha}{n}(Y_{true}-A^{[2]}) \odot A^{[2]} \odot (1-A^{[2]})\right) \bullet A^{[1]T}$$

**Weight Update for `w1`:**

$$W^{[1]} = W^{[1]} + \Delta W^{[1]}$$
$$\Delta W^{[1]} = - \alpha \frac{\partial cost}{\partial W^{[1]}}$$

$$\frac{\partial cost}{\partial W^{[1]}} = \left(\left(\frac{-2}{n}(Y_{true}-A^{[2]}) \odot A^{[2]} \odot (1-A^{[2]})\right)^T \bullet W^{[2]}\right)^T \odot \frac{\partial A^{[1]}}{\partial Z^{[1]}} \bullet X^T$$

$$W^{[1]} = W^{[1]} + \left(\left(\frac{2 \alpha}{n}(Y_{true}-A^{[2]}) \odot A^{[2]} \odot (1-A^{[2]})\right)^T \bullet W^{[2]}\right)^T \odot \frac{\partial A^{[1]}}{\partial Z^{[1]}} \bullet X^T$$

**Note:** The symbol $\odot$ represents element-wise multiplication, and the symbol $\bullet$ represents matrix multiplication.

To obtain the value of $\frac{\partial A^{[1]}}{\partial Z^{[1]}}$, which is the derivative of the ReLU function, use the following code snippet. This will produce a matrix of the same size as $Z^{[1]}$, composed of `0` and `1`, where cells corresponding to $Z^{[1]} > 0$ will have a value of `1`, and `0` otherwise. Note that although you pass `A_1` as input to this function, it does not affect the output.

```python
relu_gradient = np.where(A_1 > 0, 1, 0)
```

**Important:** Part of $\Delta W^{[1]}$ is already computed in $\Delta W^{[2]}$. By storing it, you can avoid redundant calculations.

#### `fit` Method

The `fit(self, inputs, outputs, learning_rate, epochs=64)` method updates the network's weights for the specified number of epochs (`epochs`). You do not need to make any changes to this method; simply use it in the subsequent steps.

### Model Class Implementation

In [65]:
class Model:

    def __init__(self):
        self.w1 = np.random.randn(1000, X_train.shape[0]) * 0.01
        self.w2 = np.random.randn(1, 1000) * 0.01

    def predict(self, inputs):
        x = inputs

        Z_1 = np.dot(self.w1, x)
        A_1 = np.maximum(0, Z_1)

        Z_2 = np.dot(self.w2, A_1)
        A_2 = 1 / (1 + np.exp(-Z_2))

        return A_1, A_2

    def update_weights_for_one_epoch(self, inputs, outputs, learning_rate):
        x, y_true = inputs, outputs
        A_1, A_2 = self.predict(inputs)

        n = x.shape[1]

        shared_coefficient = 2 * learning_rate / n * ((y_true - A_2) * A_2 * (1 - A_2))
        relu_gradient = np.where(A_1 > 0, 1, 0)
        
        self.w1 = self.w1 + np.dot(((np.dot(shared_coefficient.T, self.w2)).T * relu_gradient), x.T)
        self.w2 = self.w2 + np.dot(shared_coefficient, A_1.T)

    def fit(self, inputs, outputs, learning_rate, epochs=64):
        for i in range(epochs):
            self.update_weights_for_one_epoch(inputs, outputs, learning_rate)

### Training and Evaluation

After designing the network structure, you can create an instance of the `Model()` class and then call the `fit` method with appropriate arguments to start training the model. It is recommended to experiment with different learning rates (such as `0.1`, `0.01`, `0.001`, etc.) and different numbers of training epochs, and compare the results on the validation samples.

To assess the model's accuracy, you can use the `evaluation(model, inputs, outputs)` function.

In [69]:
def evaluation(model, inputs, outputs):
  _, A_2 = model.predict(inputs)
  prediction = (A_2 > 0.5)
  return np.mean(prediction == outputs) * 100

model = Model()
model.fit(X_train, y_train.to_numpy().reshape(1, -1), learning_rate = 0.02, epochs = 1000) # TODO

# Model evaluation
print(f"Your model accuracy on the given set: {round(evaluation(model, X_validation, y_validation.to_numpy().reshape(1, -1)), 2)}%") # TODO

Your model accuracy on the given set: 73.88%


## Prediction on Test Data and Output

Finally, you need to compute the model's output for the test samples. First, obtain the model's output on the test data, and then if the model predicts a higher probability that an individual has diabetes (output greater than `0.5`), classify the individual as diabetic; otherwise, classify them as non-diabetic.

Therefore, in the `prediction` variable, which is a NumPy array, you will have `True` and `False` values. Note that this variable will also be evaluated by the grading system.

In [71]:
_, output = model.predict(test_data_numpy) # TODO
prediction = (output > 0.5)
prediction

array([[False,  True,  True, False, False,  True, False,  True, False,
        False, False, False, False,  True, False, False, False, False,
        False, False, False,  True, False,  True, False,  True, False,
         True, False, False, False, False, False, False,  True, False,
        False, False, False, False,  True, False, False, False,  True,
        False, False,  True,  True, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False,  True, False, False, False, False, False, False, False,
         True, False, False,  True,  True, False,  True, False,  True,
        False, False, False, False,  True,  True, False, False, False,
        False,  True, False,  True, False, False, False, False, False,
        False]])

## Assignment Grading Procedure

The accuracy of your model on the test data, specifically the `prediction` variable, will also be evaluated, with a minimum acceptable accuracy of **65%**.

Additionally, the `test_data` DataFrame will be checked to ensure the correctness of your data normalization process.